## المقدمة

ستغطي هذه الدرس:
- ما هو استدعاء الدالة وحالات استخدامه
- كيفية إنشاء استدعاء دالة باستخدام OpenAI
- كيفية دمج استدعاء دالة في تطبيق

## أهداف التعلم

بعد إكمال هذا الدرس ستعرف كيفية وتفهم:

- الغرض من استخدام استدعاء الدالة
- إعداد استدعاء دالة باستخدام خدمة OpenAI
- تصميم استدعاءات دالة فعالة لحالة استخدام تطبيقك


## فهم استدعاءات الدوال

لهذا الدرس، نريد بناء ميزة لشركتنا الناشئة في مجال التعليم تتيح للمستخدمين استخدام روبوت دردشة للعثور على الدورات التقنية. سنوصي بدورات تناسب مستوى مهاراتهم، والدور الحالي، والتكنولوجيا التي يهتمون بها.

لإكمال ذلك، سنستخدم مزيجًا من:
 - `OpenAI` لإنشاء تجربة دردشة للمستخدم
 - `Microsoft Learn Catalog API` لمساعدة المستخدمين في العثور على الدورات بناءً على طلب المستخدم
 - `Function Calling` لأخذ استعلام المستخدم وإرساله إلى دالة لإجراء طلب API.

للبدء، دعونا ننظر لماذا نرغب في استخدام استدعاء الدوال في المقام الأول:

print("الرسائل في الطلب التالي:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # الحصول على رد جديد من GPT حيث يمكنه رؤية استجابة الدالة


print(second_response.choices[0].message)


### لماذا استدعاء الدوال

إذا كنت قد أكملت أي درس آخر في هذه الدورة، فمن المحتمل أنك تفهم قوة استخدام نماذج اللغة الكبيرة (LLMs). ونأمل أن تكون قادرًا أيضًا على رؤية بعض محدودياتها.

استدعاء الدوال هو ميزة في خدمة OpenAI مصممة لمعالجة التحديات التالية:

تنسيق الاستجابة غير المتسق:
- قبل استدعاء الدوال، كانت الاستجابات من نموذج اللغة الكبير غير منظمة وغير متسقة. كان على المطورين كتابة كود تحقق معقد للتعامل مع كل اختلاف في المخرجات.

التكامل المحدود مع البيانات الخارجية:
- قبل هذه الميزة، كان من الصعب دمج البيانات من أجزاء أخرى من التطبيق في سياق المحادثة.

من خلال توحيد تنسيقات الاستجابة وتمكين التكامل السلس مع البيانات الخارجية، يبسط استدعاء الدوال عملية التطوير ويقلل الحاجة إلى منطق تحقق إضافي.

لم يكن بإمكان المستخدمين الحصول على إجابات مثل "ما هو الطقس الحالي في ستوكهولم؟". وذلك لأن النماذج كانت محدودة بالوقت الذي تم تدريب البيانات عليه.

لننظر إلى المثال أدناه الذي يوضح هذه المشكلة:

لنفترض أننا نريد إنشاء قاعدة بيانات لبيانات الطلاب حتى نتمكن من اقتراح الدورة المناسبة لهم. أدناه لدينا وصفان لطلاب متشابهين جدًا في البيانات التي يحتويان عليها.


In [ ]:
student_1_description="Emily Johnson is a sophomore majoring in computer science at Duke University. She has a 3.7 GPA. Emily is an active member of the university's Chess Club and Debate Team. She hopes to pursue a career in software engineering after graduating."
 
student_2_description = "Michael Lee is a sophomore majoring in computer science at Stanford University. He has a 3.8 GPA. Michael is known for his programming skills and is an active member of the university's Robotics Club. He hopes to pursue a career in artificial intelligence after finishing his studies."

نريد إرسال هذا إلى نموذج لغة كبير لتحليل البيانات. يمكن استخدام هذا لاحقًا في تطبيقنا لإرسالها إلى واجهة برمجة تطبيقات أو تخزينها في قاعدة بيانات.

لننشئ طلبين متطابقين نوجه من خلالهما نموذج اللغة الكبير حول المعلومات التي نهتم بها:


نريد إرسال هذا إلى نموذج اللغة الكبير لتحليل الأجزاء المهمة لمنتجنا. لذلك يمكننا إنشاء طلبين متطابقين لتوجيه نموذج اللغة الكبير:


In [ ]:
prompt1 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_1_description}
'''


prompt2 = f'''
Please extract the following information from the given text and return it as a JSON object:

name
major
school
grades
club

This is the body of text to extract the information from:
{student_2_description}
'''


بعد إنشاء هذين الطلبين، سنرسلهما إلى نموذج اللغة الكبير باستخدام `openai.ChatCompletion`. نقوم بتخزين الطلب في متغير `messages` ونعين الدور إلى `user`. هذا لمحاكاة رسالة من مستخدم تُكتب إلى روبوت المحادثة.


In [ ]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

deployment="gpt-3.5-turbo"

: 

الآن يمكننا إرسال كلا الطلبين إلى نموذج اللغة الكبير وفحص الاستجابة التي نتلقاها.


In [ ]:
openai_response1 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt1}]
)
openai_response1.choices[0].message.content 

In [ ]:
openai_response2 = client.chat.completions.create(
 model=deployment,    
 messages = [{'role': 'user', 'content': prompt2}]
)
openai_response2.choices[0].message.content

In [ ]:
# Loading the response as a JSON object
json_response1 = json.loads(openai_response1.choices[0].message.content)
json_response1

In [ ]:
# Loading the response as a JSON object
json_response2 = json.loads(openai_response2.choices[0].message.content )
json_response2

على الرغم من أن المطالبات متشابهة والوصف متقارب، يمكننا الحصول على تنسيقات مختلفة لخاصية `Grades`.

إذا قمت بتشغيل الخلية أعلاه عدة مرات، قد يكون التنسيق `3.7` أو `3.7 GPA`.

هذا لأن نموذج اللغة الكبير يأخذ بيانات غير منظمة في شكل المطالبة المكتوبة ويعيد أيضًا بيانات غير منظمة. نحتاج إلى وجود تنسيق منظم حتى نعرف ما الذي نتوقعه عند تخزين هذه البيانات أو استخدامها.

باستخدام استدعاء الدوال، يمكننا التأكد من أننا نتلقى بيانات منظمة في الردود. عند استخدام استدعاء الدوال، لا يقوم نموذج اللغة الكبير فعليًا باستدعاء أو تشغيل أي دوال. بدلاً من ذلك، ننشئ هيكلًا ليتبعه نموذج اللغة الكبير في ردوده. ثم نستخدم تلك الردود المنظمة لمعرفة الدالة التي يجب تشغيلها في تطبيقاتنا.


![مخطط تدفق استدعاء الدالة](../../../../translated_images/ar/Function-Flow.083875364af4f4bb.webp)


يمكننا بعد ذلك أخذ ما يتم إرجاعه من الدالة وإرساله مرة أخرى إلى نموذج اللغة الكبير. ثم سيستجيب نموذج اللغة الكبير باستخدام اللغة الطبيعية للإجابة على استفسار المستخدم.


### حالات استخدام استدعاء الدوال

**استدعاء الأدوات الخارجية**  
تُعد روبوتات الدردشة ممتازة في تقديم إجابات على أسئلة المستخدمين. من خلال استخدام استدعاء الدوال، يمكن لروبوتات الدردشة استخدام رسائل المستخدمين لإكمال مهام معينة. على سبيل المثال، يمكن لطالب أن يطلب من روبوت الدردشة "إرسال بريد إلكتروني إلى مدرسي يقول إنني أحتاج إلى مزيد من المساعدة في هذا الموضوع". يمكن أن يقوم هذا باستدعاء دالة `send_email(to: string, body: string)`

**إنشاء استعلامات API أو قواعد بيانات**  
يمكن للمستخدمين العثور على المعلومات باستخدام اللغة الطبيعية التي تتحول إلى استعلام منسق أو طلب API. مثال على ذلك يمكن أن يكون مدرس يطلب "من هم الطلاب الذين أكملوا الواجب الأخير" والذي يمكن أن يستدعي دالة باسم `get_completed(student_name: string, assignment: int, current_status: string)`

**إنشاء بيانات منظمة**  
يمكن للمستخدمين أخذ كتلة نصية أو CSV واستخدام نموذج اللغة الكبير لاستخراج المعلومات المهمة منها. على سبيل المثال، يمكن لطالب تحويل مقال من ويكيبيديا عن اتفاقيات السلام لإنشاء بطاقات تعليمية ذكية. يمكن القيام بذلك باستخدام دالة تسمى `get_important_facts(agreement_name: string, date_signed: string, parties_involved: list)`


## 2. إنشاء أول استدعاء دالة لك

تتضمن عملية إنشاء استدعاء دالة 3 خطوات رئيسية:
1. استدعاء واجهة برمجة تطبيقات إكمال الدردشة مع قائمة دوالك ورسالة المستخدم
2. قراءة استجابة النموذج لأداء إجراء مثل تنفيذ دالة أو استدعاء واجهة برمجة تطبيقات
3. إجراء استدعاء آخر لواجهة برمجة تطبيقات إكمال الدردشة مع الاستجابة من دالتك لاستخدام تلك المعلومات لإنشاء رد للمستخدم.


![تدفق استدعاء دالة](../../../../translated_images/ar/LLM-Flow.3285ed8caf4796d7.webp)


### عناصر استدعاء الدالة

#### إدخال المستخدم

الخطوة الأولى هي إنشاء رسالة للمستخدم. يمكن تعيين هذه القيمة ديناميكيًا عن طريق أخذ قيمة من إدخال نصي أو يمكنك تعيين قيمة هنا. إذا كانت هذه هي المرة الأولى التي تعمل فيها مع واجهة برمجة تطبيقات إكمال الدردشة، نحتاج إلى تحديد `role` و `content` للرسالة.

يمكن أن يكون `role` إما `system` (إنشاء القواعد)، أو `assistant` (النموذج) أو `user` (المستخدم النهائي). لاستدعاء الدالة، سنعين هذا كـ `user` وسؤال مثال.


In [ ]:
messages= [ {"role": "user", "content": "Find me a good course for a beginner student to learn Azure."} ]

### إنشاء الدوال.

بعد ذلك سنقوم بتعريف دالة ومعاملات تلك الدالة. سنستخدم دالة واحدة فقط هنا تسمى `search_courses` ولكن يمكنك إنشاء دوال متعددة.

**مهم**: يتم تضمين الدوال في رسالة النظام إلى نموذج اللغة الكبير وسيتم احتسابها ضمن عدد الرموز المتاحة لديك.


In [ ]:
functions = [
   {
      "name":"search_courses",
      "description":"Retrieves courses from the search index based on the parameters provided",
      "parameters":{
         "type":"object",
         "properties":{
            "role":{
               "type":"string",
               "description":"The role of the learner (i.e. developer, data scientist, student, etc.)"
            },
            "product":{
               "type":"string",
               "description":"The product that the lesson is covering (i.e. Azure, Power BI, etc.)"
            },
            "level":{
               "type":"string",
               "description":"The level of experience the learner has prior to taking the course (i.e. beginner, intermediate, advanced)"
            }
         },
         "required":[
            "role"
         ]
      }
   }
]

**التعريفات**

هيكل تعريف الدالة يحتوي على مستويات متعددة، كل منها له خصائصه الخاصة. إليك تفصيل الهيكل المتداخل:

**خصائص الدالة على المستوى الأعلى:**

`name` - اسم الدالة التي نريد استدعاؤها.

`description` - هذا هو وصف كيفية عمل الدالة. هنا من المهم أن تكون محددًا وواضحًا.

`parameters` - قائمة بالقيم والصيغة التي تريد من النموذج إنتاجها في استجابته.

**خصائص كائن المعلمات:**

`type` - نوع بيانات كائن المعلمات (عادةً "object").

`properties` - قائمة بالقيم المحددة التي سيستخدمها النموذج في استجابته.

**خصائص المعلمة الفردية:**

`name` - معرف ضمنيًا بواسطة مفتاح الخاصية (مثل "role"، "product"، "level").

`type` - نوع بيانات هذه المعلمة المحددة (مثل "string"، "number"، "boolean").

`description` - وصف المعلمة المحددة.

**الخصائص الاختيارية:**

`required` - مصفوفة تسرد المعلمات المطلوبة لإكمال استدعاء الدالة.


### إجراء استدعاء الدالة  
بعد تعريف دالة، نحتاج الآن إلى تضمينها في استدعاء واجهة برمجة تطبيقات إكمال الدردشة. نقوم بذلك عن طريق إضافة `functions` إلى الطلب. في هذه الحالة `functions=functions`.  

هناك أيضًا خيار لتعيين `function_call` إلى `auto`. هذا يعني أننا سنترك نموذج اللغة الكبير يقرر أي دالة يجب استدعاؤها بناءً على رسالة المستخدم بدلاً من تعيينها بأنفسنا.


In [ ]:
response = client.chat.completions.create(model=deployment, 
                                        messages=messages,
                                        functions=functions, 
                                        function_call="auto") 

print(response.choices[0].message)

الآن دعونا نلقي نظرة على الاستجابة ونرى كيف تم تنسيقها:

{
  "role": "assistant",
  "function_call": {
    "name": "search_courses",
    "arguments": "{\n  \"role\": \"student\",\n  \"product\": \"Azure\",\n  \"level\": \"beginner\"\n}"
  }
}

يمكنك أن ترى أن اسم الدالة تم استدعاؤه ومن رسالة المستخدم، تمكن نموذج اللغة الكبير من العثور على البيانات لتناسب معطيات الدالة.


## 3. دمج استدعاءات الدوال في تطبيق.

بعد أن قمنا باختبار الاستجابة المنسقة من نموذج اللغة الكبير، يمكننا الآن دمج هذا في تطبيق.

### إدارة التدفق

لدمج هذا في تطبيقنا، دعونا نتبع الخطوات التالية:

أولاً، دعونا نقوم بالاتصال بخدمات OpenAI وتخزين الرسالة في متغير يسمى `response_message`.


In [ ]:
response_message = response.choices[0].message

الآن سنقوم بتعريف الدالة التي ستستدعي واجهة برمجة تطبيقات Microsoft Learn للحصول على قائمة الدورات:


In [ ]:
import requests

def search_courses(role, product, level):
    url = "https://learn.microsoft.com/api/catalog/"
    params = {
        "role": role,
        "product": product,
        "level": level
    }
    response = requests.get(url, params=params)
    modules = response.json()["modules"]
    results = []
    for module in modules[:5]:
        title = module["title"]
        url = module["url"]
        results.append({"title": title, "url": url})
    return str(results)



كممارسة مثلى، سنرى بعد ذلك ما إذا كان النموذج يرغب في استدعاء دالة. بعد ذلك، سننشئ واحدة من الدوال المتاحة ونتطابق معها الدالة التي يتم استدعاؤها.  
ثم سنأخذ معاملات الدالة ونربطها بالمعاملات من LLM.

أخيرًا، سنضيف رسالة استدعاء الدالة والقيم التي تم إرجاعها بواسطة رسالة `search_courses`. هذا يمنح LLM كل المعلومات التي يحتاجها للرد على المستخدم باستخدام اللغة الطبيعية.


In [ ]:
# Check if the model wants to call a function
if response_message.function_call.name:
    print("Recommended Function call:")
    print(response_message.function_call.name)
    print()

    # Call the function. 
    function_name = response_message.function_call.name

    available_functions = {
            "search_courses": search_courses,
    }
    function_to_call = available_functions[function_name] 

    function_args = json.loads(response_message.function_call.arguments)
    function_response = function_to_call(**function_args)

    print("Output of function call:")
    print(function_response)
    print(type(function_response))


    # Add the assistant response and function response to the messages
    messages.append( # adding assistant response to messages
        {
            "role": response_message.role,
            "function_call": {
                "name": function_name,
                "arguments": response_message.function_call.arguments,
            },
            "content": None
        }
    )
    messages.append( # adding function response to messages
        {
            "role": "function",
            "name": function_name,
            "content":function_response,
        }
    )



الآن سنرسل الرسالة المحدثة إلى نموذج اللغة الكبير (LLM) حتى نتمكن من تلقي رد بلغة طبيعية بدلاً من رد منسق بصيغة JSON الخاصة بواجهة برمجة التطبيقات.


In [ ]:
print("Messages in next request:")
print(messages)
print()

second_response = client.chat.completions.create(
    messages=messages,
    model=deployment,
    function_call="auto",
    functions=functions,
    temperature=0
        )  # get a new response from GPT where it can see the function response


print(second_response.choices[0].message)

## تحدي الكود

عمل رائع! لمواصلة تعلمك لاستدعاء دوال OpenAI يمكنك بناء: https://learn.microsoft.com/training/support/catalog-api-developer-reference?WT.mc_id=academic-105485-koreyst  
 - المزيد من معلمات الدالة التي قد تساعد المتعلمين في العثور على المزيد من الدورات. يمكنك العثور على معلمات API المتاحة هنا:  
 - إنشاء استدعاء دالة آخر يأخذ المزيد من المعلومات من المتعلم مثل لغته الأم  
 - إنشاء معالجة للأخطاء عندما لا يعيد استدعاء الدالة و/أو استدعاء API أي دورات مناسبة


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**إخلاء المسؤولية**:  
تمت ترجمة هذا المستند باستخدام خدمة الترجمة الآلية [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو عدم دقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق به. للمعلومات الهامة، يُنصح بالترجمة البشرية المهنية. نحن غير مسؤولين عن أي سوء فهم أو تفسير ناتج عن استخدام هذه الترجمة.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
